# Edit Distance between two words

## 1. Confusion Matrix input from file

In [205]:
confusion_matrix = []
with open('confusion_matrix.txt','r') as file:      
    for line in file:     
        temp = []
        for number in line.split():   
            temp.append(int(number))
        confusion_matrix.append(temp)

## 2. Function to create Edit Distance matrix and Direction Matrix

+ Edit distance matrix and Direction matrix are filled using dynamic programming
+ Edit Distance matrix represent edit distance between substrings $incorrect(0,i)$ and $correct(0,j)$
+ Edit Distance Matrix : $ D[i][j] = min(D(i-1,j)+1,D(i,j-1)+1,D(i-1,j-1)+cost) $ where
    + if incorrect[i]=correcr[j] then $ cost = 0 $
    + else $ cost = ConfusionMatrix[incorrect[i]][correct[j]] $
+ Direction Matrix : Direction[i][j] contains tuple (row,column) for which minimum of D[i][j] was found 
+ Direction matrix will later be used to backtrace the path to convert incorrect word to correct word

In [206]:
def get_edit_distance(incorrect,correct):
    cols = len(correct)
    rows = len(incorrect)
    D = [[0 for x in range(cols+1)] for y in range(rows+1)]
    Direction =  [[0 for x in range(cols+1)] for y in range(rows+1)]
    #if correct string is empty, all the characters of incorrect words need to be deleted
    #all the direction(i,0) point upwards to delete characters of incorrect word
    for i in range(rows+1):
        D[i][0] = i
        Direction[i][0] = (i-1,0,1)
    #if incorrect string is empty, all the characters of correct words need to be inserted to incorrect word
    #all the direction(0,i) point towards left to insert characters correct words into incorrect word
    for i in range(cols+1):
        D[0][i] = i
        Direction[0][i] = (0,i-1,1)
    Direction[0][0]=None
    for i in range(1,rows+1):
        for j in range(1,cols+1):
            #min_arr stores the the tuple (computed_cost,(row,column,operation_cost)) computed for D(i-1,j),D(i,j-1),D(i-1,j-1) 
            #then minimum amongst these is allotted to D(i,j) and Direction(i,j)
            min_arr = []
            #deletion with cost=1 
            min_arr.append((D[i-1][j]+1,(i-1,j,1)))
            #insertion with cost=1
            min_arr.append((D[i][j-1]+1,(i,j-1,1)))
            #if same characters, replacement cost is 0
            if incorrect[i-1]==correct[j-1]:
                min_arr.append((D[i-1][j-1],(i-1,j-1,0)))
            else :
                #replacement cost for different characters is found by using confusion matrix
                #by looking at row of incorrect character and column of correct character 
                operation_cost=confusion_matrix[ord(incorrect[i-1]) - 97][ord(correct[j-1]) - 97]
                min_arr.append((D[i-1][j-1] + operation_cost,(i-1,j-1,operation_cost)))
            #sort the min_arr according to all computed_costs for D(i,j)
            min_arr.sort(key = lambda x: x[0])
            D[i][j] = min_arr[0][0]
            Direction[i][j] = min_arr[0][1]
    return D[rows][cols],D,Direction

## 3. Backtracing the path using Direction 

In [212]:
def backtrace_path(incorrect,correct,D,Direction):
    cols = len(correct)
    rows = len(incorrect)
    if rows<1 and cols<1:
        print("Since both the words are empty, no backtracing required")
        return
    curr_row,curr_col = rows,cols
    prev_row,prev_col,operation_cost =  Direction[curr_row][curr_col]
    while(1):
        # check where prev_row,prev_col lies with respect to curr_row,curr_col
        if prev_row==curr_row-1 and prev_col == curr_col-1:
            incorrect_char = incorrect[curr_row-1]
            correct_char = correct[curr_col-1]
            if incorrect_char == correct_char:
                print("Same character(",correct_char,") \tCost : 0")
            else:
                print("Replace",incorrect_char,"->",correct_char," \tCost :",operation_cost)
        elif prev_col==curr_col-1 and prev_row==curr_row:
            correct_char = correct[curr_col-1]
            print("Insert :",correct_char," \t\tCost : 1")
        elif prev_row==curr_row-1 and prev_col==curr_col:
            incorrect_char = incorrect[curr_row-1]
            print("Delete :",incorrect_char," \t\tCost : 1")
        curr_row,curr_col=prev_row,prev_col
        #cgeck if Direction[0][0] is reached
        if Direction[prev_row][prev_col]==None:
            break
        prev_row,prev_col,operation_cost =  Direction[prev_row][prev_col]
    

## 4. Input of correct and incorrect word

### 4.1 Both incorrect and correct words are empty

In [208]:
incorrect = input("Enter incorrect word:")
correct = input("Enter correct word to find minimum edit distance:")
distance,D,Direction = get_edit_distance(incorrect,correct)
print('-------------------------------------------------------------------------------------------------------------------------------')
print("Edit Distance between",incorrect,"and",correct,"is :",distance)
print('-------------------------------------------------------------------------------------------------------------------------------')
backtrace_path(incorrect,correct,D,Direction)

Enter incorrect word:
Enter correct word to find minimum edit distance:
-------------------------------------------------------------------------------------------------------------------------------
Edit Distance between  and  is : 0
-------------------------------------------------------------------------------------------------------------------------------
Since both the words are empty, no backtracing required


### 4.2 Correct word is empty

In [209]:
incorrect = input("Enter incorrect word:")
correct = input("Enter correct word to find minimum edit distance:")
distance,D,Direction = get_edit_distance(incorrect,correct)
print('-------------------------------------------------------------------------------------------------------------------------------')
print("Edit Distance between",incorrect,"and",correct,"is :",distance)
print('-------------------------------------------------------------------------------------------------------------------------------')
backtrace_path(incorrect,correct,D,Direction)

Enter incorrect word:intention
Enter correct word to find minimum edit distance:
-------------------------------------------------------------------------------------------------------------------------------
Edit Distance between intention and  is : 9
-------------------------------------------------------------------------------------------------------------------------------
Delete : n  		Cost : 1
Delete : o  		Cost : 1
Delete : i  		Cost : 1
Delete : t  		Cost : 1
Delete : n  		Cost : 1
Delete : e  		Cost : 1
Delete : t  		Cost : 1
Delete : n  		Cost : 1
Delete : i  		Cost : 1


### 4.3 Incorrect word is empty

In [210]:
incorrect = input("Enter incorrect word:")
correct = input("Enter correct word to find minimum edit distance:")
distance,D,Direction = get_edit_distance(incorrect,correct)
print('-------------------------------------------------------------------------------------------------------------------------------')
print("Edit Distance between",incorrect,"and",correct,"is :",distance)
print('-------------------------------------------------------------------------------------------------------------------------------')
backtrace_path(incorrect,correct,D,Direction)

Enter incorrect word:
Enter correct word to find minimum edit distance:intention
-------------------------------------------------------------------------------------------------------------------------------
Edit Distance between  and intention is : 9
-------------------------------------------------------------------------------------------------------------------------------
Insert : n  		Cost : 1
Insert : o  		Cost : 1
Insert : i  		Cost : 1
Insert : t  		Cost : 1
Insert : n  		Cost : 1
Insert : e  		Cost : 1
Insert : t  		Cost : 1
Insert : n  		Cost : 1
Insert : i  		Cost : 1


### 4.4 Both incorrect and correct words are not empty

### Example 1

In [211]:
incorrect = input("Enter incorrect word:")
correct = input("Enter correct word to find minimum edit distance:")
distance,D,Direction = get_edit_distance(incorrect,correct)
print('-------------------------------------------------------------------------------------------------------------------------------')
print("Edit Distance between",incorrect,"and",correct,"is :",distance)
print('-------------------------------------------------------------------------------------------------------------------------------')
backtrace_path(incorrect,correct,D,Direction)

Enter incorrect word:intention
Enter correct word to find minimum edit distance:execution
-------------------------------------------------------------------------------------------------------------------------------
Edit Distance between intention and execution is : 4
-------------------------------------------------------------------------------------------------------------------------------
Same character( n ) 	Cost : 0
Same character( o ) 	Cost : 0
Same character( i ) 	Cost : 0
Same character( t ) 	Cost : 0
Replace n -> u  	Cost : 0
Insert : c  		Cost : 1
Same character( e ) 	Cost : 0
Replace t -> x  	Cost : 0
Delete : n  		Cost : 1
Delete : i  		Cost : 1
Insert : e  		Cost : 1


### Example 2

In [218]:
incorrect = input("Enter incorrect word:")
correct = input("Enter correct word to find minimum edit distance:")
distance,D,Direction = get_edit_distance(incorrect,correct)
print('-------------------------------------------------------------------------------------------------------------------------------')
print("Edit Distance between",incorrect,"and",correct,"is :",distance)
print('-------------------------------------------------------------------------------------------------------------------------------')
backtrace_path(incorrect,correct,D,Direction)

Enter incorrect word:acceleration
Enter correct word to find minimum edit distance:deceleration
-------------------------------------------------------------------------------------------------------------------------------
Edit Distance between acceleration and deceleration is : 1
-------------------------------------------------------------------------------------------------------------------------------
Same character( n ) 	Cost : 0
Same character( o ) 	Cost : 0
Same character( i ) 	Cost : 0
Same character( t ) 	Cost : 0
Same character( a ) 	Cost : 0
Same character( r ) 	Cost : 0
Same character( e ) 	Cost : 0
Same character( l ) 	Cost : 0
Same character( e ) 	Cost : 0
Same character( c ) 	Cost : 0
Replace c -> e  	Cost : 0
Replace a -> d  	Cost : 1
